# UDP

<img src='res/udp02.png' width=500px />

<img src='res/udp01.png' width=500px />

1. Hash table to store udp socks. key is the port `number & (UDP_HTABLE_SIZE - 1)`

### static struct sock *udp_v4_lookup_longway

```c
/* UDP is nearly always wildcards out the wazoo, it makes no sense to try
 * harder than this. -DaveM
 */
static struct sock *udp_v4_lookup_longway(u32 saddr, u16 sport,
					  u32 daddr, u16 dport, int dif)
{
	struct sock *sk, *result = NULL;
	struct hlist_node *node;
	unsigned short hnum = ntohs(dport);
	int badness = -1;

	sk_for_each(sk, node, &udp_hash[hnum & (UDP_HTABLE_SIZE - 1)]) {
		struct inet_sock *inet = inet_sk(sk);

		if (inet->num == hnum && !ipv6_only_sock(sk)) {
			int score = (sk->sk_family == PF_INET ? 1 : 0);
			if (inet->rcv_saddr) {
				if (inet->rcv_saddr != daddr)
					continue;
				score+=2;
			}
			if (inet->daddr) {
				if (inet->daddr != saddr)
					continue;
				score+=2;
			}
			if (inet->dport) {
				if (inet->dport != sport)
					continue;
				score+=2;
			}
			if (sk->sk_bound_dev_if) {
				if (sk->sk_bound_dev_if != dif)
					continue;
				score+=2;
			}
			/* xitongsys
			score == 9，所有条件都满足，直接返回
			*/
			if(score == 9) { 
				result = sk;
				break;
			} else if(score > badness) { //xitongsys： 找到匹配最多（score最大）的sock
				result = sk;
				badness = score;
			}
		}
	}
	return result;
}
```

1. 从hash表中查找匹配的sock。因为一个port可以绑定多个sock对象，这里采用了最有匹配。

### static int udp_v4_get_port(struct sock *sk, unsigned short snum)

```c
/* Shared by v4/v6 udp. */
int udp_port_rover;

static int udp_v4_get_port(struct sock *sk, unsigned short snum)
{
	struct hlist_node *node;
	struct sock *sk2;
	struct inet_sock *inet = inet_sk(sk);

	write_lock_bh(&udp_hash_lock);
	if (snum == 0) {
		int best_size_so_far, best, result, i;

		if (udp_port_rover > sysctl_local_port_range[1] ||
		    udp_port_rover < sysctl_local_port_range[0])
			udp_port_rover = sysctl_local_port_range[0];
		best_size_so_far = 32767;
		best = result = udp_port_rover;
		for (i = 0; i < UDP_HTABLE_SIZE; i++, result++) {
			struct hlist_head *list;
			int size;

			list = &udp_hash[result & (UDP_HTABLE_SIZE - 1)];
			if (hlist_empty(list)) {
				if (result > sysctl_local_port_range[1])
					result = sysctl_local_port_range[0] +
						((result - sysctl_local_port_range[0]) &
						 (UDP_HTABLE_SIZE - 1));
				goto gotit;
			}
			size = 0;
			sk_for_each(sk2, node, list)
				if (++size >= best_size_so_far)
					goto next;
			best_size_so_far = size;
			best = result;
		next:;
		}
		result = best;
		for(i = 0; i < (1 << 16) / UDP_HTABLE_SIZE; i++, result += UDP_HTABLE_SIZE) {
			if (result > sysctl_local_port_range[1])
				result = sysctl_local_port_range[0]
					+ ((result - sysctl_local_port_range[0]) &
					   (UDP_HTABLE_SIZE - 1));
			if (!udp_lport_inuse(result))
				break;
		}
		if (i >= (1 << 16) / UDP_HTABLE_SIZE)
			goto fail;
gotit:
		udp_port_rover = snum = result;
	} else {
		sk_for_each(sk2, node,
			    &udp_hash[snum & (UDP_HTABLE_SIZE - 1)]) {
			struct inet_sock *inet2 = inet_sk(sk2);

			if (inet2->num == snum &&
			    sk2 != sk &&
			    !ipv6_only_sock(sk2) &&
			    (!sk2->sk_bound_dev_if ||
			     !sk->sk_bound_dev_if ||
			     sk2->sk_bound_dev_if == sk->sk_bound_dev_if) &&
			    (!inet2->rcv_saddr ||
			     !inet->rcv_saddr ||
			     inet2->rcv_saddr == inet->rcv_saddr) &&
			    (!sk2->sk_reuse || !sk->sk_reuse))
				goto fail;
		}
	}
	inet->num = snum;
	if (sk_unhashed(sk)) {
		struct hlist_head *h = &udp_hash[snum & (UDP_HTABLE_SIZE - 1)];

		sk_add_node(sk, h);
		sock_prot_inc_use(sk->sk_prot);
	}
	write_unlock_bh(&udp_hash_lock);
	return 0;

fail:
	write_unlock_bh(&udp_hash_lock);
	return 1;
}
```

1. snum=0, 获取一个可用的port，在sysctl_local_port_range[0]，sysctl_local_port_range[1] 之间。否则就检查snum是否可用

<img src='res/udp03.png' width=400px />

2. 回去遍历每个hash slot里面list的长度，找到最短的那个作port

### int udp_rcv(struct sk_buff *skb)

```c
/*
 *	All we need to do is get the socket, and then do a checksum. 
 */
 
int udp_rcv(struct sk_buff *skb)
{
    ...

	if(rt->rt_flags & (RTCF_BROADCAST|RTCF_MULTICAST))
		return udp_v4_mcast_deliver(skb, uh, saddr, daddr);

	sk = udp_v4_lookup(saddr, uh->source, daddr, uh->dest, skb->dev->ifindex);

	if (sk != NULL) {
		int ret = udp_queue_rcv_skb(sk, skb);
		sock_put(sk);

		/* a return value > 0 means to resubmit the input, but
		 * it it wants the return to be -protocol, or 0
		 */
		if (ret > 0)
			return -ret;
		return 0;
	}

    ...

}
```

1. 这个函数就是inet_protocols里面udp协议的handler。就是根据收到的sk_buff找到对应的socket，交由sock处理

### static int udp_queue_rcv_skb(struct sock * sk, struct sk_buff *skb)
### static inline int sock_queue_rcv_skb(struct sock *sk, struct sk_buff *skb)

```c
static inline int sock_queue_rcv_skb(struct sock *sk, struct sk_buff *skb)
{
    ...
	skb_queue_tail(&sk->sk_receive_queue, skb);
}
```

这两个函数将收到的skb，放到对应的sock的receive queue里

### static int udp_recvmsg(struct kiocb *iocb, struct sock *sk, struct msghdr *msg,

--------------
--------------

## net/core/datagram.c

### struct sk_buff *skb_recv_datagram(struct sock *sk, unsigned flags,struct sk_buff *skb_recv_datagram(struct sock *sk, unsigned flags,

```c
struct sk_buff *skb_recv_datagram(struct sock *sk, unsigned flags,
				  int noblock, int *err)
{
	struct sk_buff *skb;
	long timeo;
	/*
	 * Caller is allowed not to check sk->sk_err before skb_recv_datagram()
	 */
	int error = sock_error(sk);

	if (error)
		goto no_packet;

	timeo = sock_rcvtimeo(sk, noblock);

	do {
		/* Again only user level code calls this function, so nothing
		 * interrupt level will suddenly eat the receive_queue.
		 *
		 * Look at current nfs client by the way...
		 * However, this function was corrent in any case. 8)
		 */
		if (flags & MSG_PEEK) {
			unsigned long cpu_flags;

			spin_lock_irqsave(&sk->sk_receive_queue.lock,
					  cpu_flags);
			skb = skb_peek(&sk->sk_receive_queue);
			if (skb)
				atomic_inc(&skb->users);
			spin_unlock_irqrestore(&sk->sk_receive_queue.lock,
					       cpu_flags);
		} else
			skb = skb_dequeue(&sk->sk_receive_queue);

		if (skb)
			return skb;

		/* User doesn't want to wait */
		error = -EAGAIN;
		if (!timeo)
			goto no_packet;

	} while (!wait_for_packet(sk, err, &timeo));

	return NULL;

no_packet:
	*err = error;
	return NULL;
}
```

1. 从sock的receive_queue里 dequeue/peek一个skb。如果没有，error=EAGAIN
2. skb_dequeue 本身内部有lock，而skb_peek是没有lock，所以这里单独加锁了
```c
/**
 *	skb_peek
 *	@list_: list to peek at
 *
 *	Peek an &sk_buff. Unlike most other operations you _MUST_
 *	be careful with this one. A peek leaves the buffer on the
 *	list and someone else may run off with it. You must hold
 *	the appropriate locks or have a private queue to do this.
 *
 *	Returns %NULL for an empty list or a pointer to the head element.
 *	The reference count is not incremented and the reference is therefore
 *	volatile. Use with caution.
 */
static inline struct sk_buff *skb_peek(struct sk_buff_head *list_)
{
	struct sk_buff *list = ((struct sk_buff *)list_)->next;
	if (list == (struct sk_buff *)list_)
		list = NULL;
	return list;
}

/**
 *	skb_dequeue - remove from the head of the queue
 *	@list: list to dequeue from
 *
 *	Remove the head of the list. The list lock is taken so the function
 *	may be used safely with other locking list functions. The head item is
 *	returned or %NULL if the list is empty.
 */

struct sk_buff *skb_dequeue(struct sk_buff_head *list)
{
	unsigned long flags;
	struct sk_buff *result;

	spin_lock_irqsave(&list->lock, flags);
	result = __skb_dequeue(list);
	spin_unlock_irqrestore(&list->lock, flags);
	return result;
}

```